# Peng-Robinson Equation of State

In [1]:
import numpy as np
import pandas as pd
import sympy as sp
from sympy import *
import math

In [42]:
#Input Pressure and Temperature values
P = float(input("Enter pressure in psia:")) # psia
T = float(input("Enter Temperature in Fahrenheit: ")) # Fahrenheit

print(f'For Pressure: {P} psia and Temperature: {T} Fahrenheit' )

Enter pressure in psia:500
Enter Temperature in Fahrenheit: 70
For Pressure: 500.0 psia and Temperature: 70.0 Fahrenheit


In [43]:
#Import Components Properties
component_properties = pd.read_excel("components.xlsx")
component_properties
# Pci = critical pressure
# Tci = critical temperature
# wi = accentric factor
# Mwi = molecular weight
# Vci = critical volume

,component,composition,Pci,Tci,wi,Mwi,Vci
0,N2,0.0200,493.1,227.49,0.040,28.013,0.0510
1,C1,0.8866,666.4,343.33,0.008,16.043,0.0988
2,C2,0.0492,706.5,549.92,0.098,30.070,0.0783
3,C3,0.0246,616.0,666.06,0.152,44.097,0.0727
4,nC4,0.0098,550.6,765.62,0.193,58.123,0.0703
5,nC5,0.0098,488.6,845.80,0.251,72.151,0.0675


In [28]:
#Import binary coefficients
d= pd.read_excel("binary.xlsx").to_numpy()
d

array([[0.    , 0.018 , 0.039 , 0.046 , 0.047 , 0.048 ],
       [0.018 , 0.    , 0.005 , 0.01  , 0.0145, 0.0182],
       [0.039 , 0.005 , 0.    , 0.0017, 0.0032, 0.0048],
       [0.046 , 0.01  , 0.0017, 0.    , 0.0012, 0.0024],
       [0.047 , 0.0145, 0.0032, 0.0012, 0.    , 0.0008],
       [0.048 , 0.0182, 0.0048, 0.0024, 0.0008, 0.    ]])

In [44]:
Pci = component_properties['Pci'] 
Tci = component_properties['Tci']

Pri= P/Pci
Tri= (T+459.67 )/Tci # -459.67 = absolute zero

print('Reduced Pressure,  Pri:\n', Pri)
print('Reduced Temperature, Tri:\n',Tri)

Reduced Pressure,  Pri:
 0    1.013993
1    0.750300
2    0.707714
3    0.811688
4    0.908100
5    1.023332
Name: Pci, dtype: float64
Reduced Temperature, Tri:
 0    2.328322
1    1.542743
2    0.963176
3    0.795229
4    0.691818
5    0.626236
Name: Tci, dtype: float64


In [45]:
wi = component_properties['wi'] #accentric factor

w_a = 0.45723553
w_b = 0.07779607

for i in wi:
    if i<0.49:
        mi = 0.3796+1.54226*wi-0.2699*wi**2
    else:
        mi = 0.379642+1.48503*wi-0.164423*wi**2+0.016667*wi**3
        
Ai =(w_a*(1+mi*(1-Tri**0.5))**2*(Pri/Tri**2)).replace(np.nan,0).to_numpy()
Bi = ((w_b*Pri)/Tri).to_numpy()
print('Ai:',Ai)
print('Bi:',Bi)

Ai: [0.05046516 0.11808825 0.35568868 0.66663486 1.07323913 1.59557147]
Bi: [0.03388048 0.03783546 0.05716229 0.07940629 0.10211731 0.12712662]


In [46]:
#Calculating A and B

c = component_properties['composition'].to_numpy()
n= len(c)

B =  np.sum(c * Bi)

A=0
for i in range (n):
    for j in range (n):
        A += (1-d[i,j])*(Ai[i]*Ai[j])**0.5*c[i]*c[j] 
        
print('A :', A)
print('B :', B)

A : 0.14472949394965842
B : 0.041234901215613226


In [47]:
#Calculating Z 

m1 = 1+2**0.5
m2 = 1-2**0.5
p=1
q=(m1+m2-1)*B-1
r=(A+m1*m2*B**2-(m1+m2)*B*(B+1))
s=-A*B-m1*m2*B**2*(B+1)

Z = np.roots([p, q, r, s])

print(f'For Pressure: {P} psia  and Temperature: {T} Farenheit' )
print('                          A:', round(A,4))
print('                          B:', round(B,4))
print('                          Z:', Z)        
print('                      Z_max:', max(np.real(Z)))
print('                      Z_min:', min(np.real(Z)))

For Pressure: 500.0 psia  and Temperature: 70.0 Farenheit
                          A: 0.1447
                          B: 0.0412
                          Z: [0.9004649+0.j         0.0291501+0.06173918j 0.0291501-0.06173918j]
                      Z_max: 0.9004648972943421
                      Z_min: 0.02915010074502255


# Wilsons Correlation, K-values

In [48]:
Ki = (1/Pri)*((5.37*(1+wi)*(1-(1/Tri))).apply(math.exp))

Ki 

0    23.861510
1     8.949366
2     1.127827
3     0.250495
4     0.063454
5     0.017729
dtype: float64

# Composition of Liquid and Vapour phase

In [49]:
g_V = lambda v:sum(c*(Ki - 1)/(v*(Ki - 1)+1))
dg_V = lambda v:-sum(c*((Ki-1)**2)/((v*(Ki-1) + 1)**2))

A = sum(c*(Ki-1))
B = sum(c*(Ki-1)/Ki)

v = A/(A - B) 

while abs(g_V(v))>0.0001:
    v = v - g_V(v)/dg_V(v)

l = 1 - v

print(f"Composition in vapor phase is: {v}")
print(f"Composition in liquid phase is: {l}")

Composition in vapor phase is: 1.00142788665634
Composition in liquid phase is: -0.0014278866563399717


# Calculation of A and B in Liquid and Vapour phase

In [50]:
Xi = c/(1 + v*(Ki-1))
Yi = Xi*Ki

def Aa(c):
    return [sum(c[i]*c*(Ai[i]*Ai)**0.5) for i in range(len(c))]

Al = sum(Aa(Xi))
Av = sum(Aa(Yi))
Bl = sum(Xi*Bi)
Bv = sum(Yi*Bi)

print(f"A in Vapour phase {Av} and liquid phase {Al}")
print(f"B in Vapour phase {Bv} and liquid phase {Bl}")

A in Vapour phase 0.14599643691793535 and liquid phase 1.128711704174488
B in Vapour phase 0.04132798503624743 and liquid phase 0.10651790765434722


In [51]:
# m1 = 1+2**0.5
# m2 = 1-2**0.5

# #Vapour Phase 
# p=1
# q=(m1+m2-1)*Bv-1
# r=(Av+m1*m2*Bv**2-(m1+m2)*Bv*(Bv+1))
# s=-Av*Bv-m1*m2*Bv**2*(Bv+1)

# Zv = np.roots([p, q, r, s])

# print('Vapour Phase compressibility factor, Zv:', np.real(Zv))

# Zv = max(Zv)
# print('Vapour Phase :', np.real(Zv))

# #Liquid phase 
# t=1
# u=(m1+m2-1)*Bl-1
# w=(Al+m1*m2*Bl**2-(m1+m2)*Bl*(Bl+1))
# x=-Al*Bl-m1*m2*Bl**2*(Bl+1)

# Zl = np.roots([t, u, w, x])

# for i in range(len(Zl)):
#     if np.isreal(Zl[i]):
#         print('Liquid Phase compressibility factor, Zl:', np.real(Zl[i]))

# Compressibility of Liquid and Vapour phase

In [52]:
Zl, Zv = symbols("Zl Zv")

f_Zl = Zl**3+(Bl-1)*Zl**2+(Al-3*(Bl)**2-2*Bl)*Zl-(Al*Bl-(Bl)**2-(Bl)**3)
f_Zv = Zv**3+(Bv-1)*Zv**2+(Av-3*(Bv)**2-2*Bv)*Zv-(Av*Bv-(Bv)**2-(Bv)**3)

Zl_solutions = solve(f_Zl, Zl)
Zv_solutions = solve(f_Zv, Zv)

# eliminate complex roots:
Zl_real_solutions = [sol.evalf() for sol in Zl_solutions if im(sol) == 0]
Zv_real_solutions = [sol.evalf() for sol in Zv_solutions if im(sol) == 0]

if Zl_real_solutions:
    Zl = min(Zl_real_solutions)
    print(f"Liquid Phase compressibility factor, Zl: {Zl}")
else:
    print("No real solutions for Liquid Phase compressibility factor")

if Zv_real_solutions:
    Zv = max(Zv_real_solutions)
    print(f"Vapor Phase compressibility factor, Zv: {Zv}")
else:
    print("No real solutions for Vapor Phase compressibility factor")


Liquid Phase compressibility factor, Zl: 0.138570583988524
Vapor Phase compressibility factor, Zv: 0.899191626494722


# Results

In [53]:
print(f'For Pressure: {P} psia  and Temperature: {T} Farenheit' )

print(f"Composition of Vapor phase : {v} and Liquid Phase: {l}")

print(f"A in Vapour phase {Av} and Liquid phase {Al}")

print(f"B in Vapour phase {Bv} and Liquid phase {Bl}")

print(f"Compressibility factor,  Vapour phase , Zv: {Zv} and Liquid phase, Zl: {Zl} ")


For Pressure: 500.0 psia  and Temperature: 70.0 Farenheit
Composition of Vapor phase : 1.00142788665634 and Liquid Phase: -0.0014278866563399717
A in Vapour phase 0.14599643691793535 and Liquid phase 1.128711704174488
B in Vapour phase 0.04132798503624743 and Liquid phase 0.10651790765434722
Compressibility factor,  Vapour phase , Zv: 0.899191626494722 and Liquid phase, Zl: 0.138570583988524 
